## Logistics

In [3]:
## Initialize environment

#!conda create -n hoaxy python=3.7
#!source activate hoaxy

In [316]:
## Import libraries and files.

import requests
import pandas as pd
import numpy as np
import os
import dotenv
from bs4 import BeautifulSoup
# !pip install selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
#!pip install xml



## Twitter files and packages.

#import recent_search
import tweepy

#!pip install python-dotenv
# !pip install tweepy
pd.set_option('display.max_rows', 25)


## Gather SEC low-credibility site domains/info

In [11]:
## Base URL for the SEC low-credibility source list

base_url="https://www.sec.gov/enforce/public-alerts"

In [27]:
## Gather links to the info for each domain

response=requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')
pattern="enforce/public-alerts/"
click_links=[]
for a in soup.find_all('a', href=True):
    if(re.search(pattern,a["href"])):
        click_links.append("https://www.sec.gov"+a["href"])

In [97]:
## For each domain info link, gather name, category, and URL for the domain

domain_name=[]
domain_category=[]
domain_website=[]

for x in click_links:
    
    # Outer loop preconditions
    
    target_url=x
    response=requests.get(target_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    target_title=soup.find("h1",class_="article-title").text
    domain_name.append(target_title)
    print(target_title)
    target_category=soup.find("h2",class_="alt-title padding-top-0").text
    domain_category.append(target_category)
    
    # Inner loop preconditions
    
    breaks=soup.find_all("br")
    found=False
    iterator=0
    for br in breaks:
        end=len(breaks)
        if(re.search("Website:",br.text)):
            try:
                target_website=re.search("(?<=Website\: ).*",br.text).group(0)
                print(target_website)
                domain_website.append(target_website)
                break
            except:
                target_website="Not found."
                print(target_website)
                domain_website.append(target_website)
                break
        if(re.search("www.",br.text)):
            try:
                target_website=re.search("(?<=www\.).*",br.text).group(0)
                print(target_website)
                domain_website.append(target_website)
                break
            except:
                target_website="Not found."
                print(target_website)
                domain_website.append(target_website)
                break
        iterator=iterator+1
        if((found==False)&(iterator==end)):
            target_website="Not found."
            print(target_website)
            domain_website.append(target_website)

12 West Capital Management
http://www.12-wcm.com
1st American Securities
http://www.firstamericansecurities.com
1st New York Financial Corporation
http://www.1stnewyorkfinancial.com    
205fx.com
 205fx.com
24 Provident Trade
www.24providenttrade.com
247 Options Trade
Not found.
247Crypto Trade
www.247cryptotrade.co 
24Brickstonefx.com
https://www.24brickstonefx.com 
24cryptoindex
https://24cryptoindex.com/index-2.html
24TradeConnect
 https://24tradeconnect.com
24TradeSpace
 http://24tradespace.com
360cryptominners
www.360cryptominners.com
360Traders
 www.360traders.co
362livefxmarket
https://362livefxmarket.com
365Tradeposition
www.365tradeposition.com
426 Investments Incorporated
www.426investmentsinc.com
9figuretradechannel
https://9figuretradechannel.com/
Abacus & Partners
www.abacusglobalgroup.com
Accer Mergers and Acquisitions
http://www.accerma.com
Access Broker.com Inc
http://www.abrokercorp.com/
Access Financial Investment
https://access-financial.com
Accrescent Holdings Inc.


In [242]:
## Check results of lists.

print(len(domain_name),len(domain_website),len(domain_category))

1448 1448 1448


In [107]:
## Create pandas table for lists and export

domains=pd.DataFrame({"company_name":domain_name, "company_website":domain_website,"sec_category":domain_category})
domains.to_csv("domains_backup.csv")

In [253]:
## Sample code for lang attribute scraping.

sample_url=domains.iloc[13].company_website
response=requests.get(sample_url)
soup = BeautifulSoup(response.content, 'html.parser')
print(sample_url)
soup.prettify()

https://362livefxmarket.com


'<!DOCTYPE html>\n<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->\n<!--[if gt IE 8]><!-->\n<html class="no-js" lang="en-US">\n <!--<![endif]-->\n <head>\n  <title>\n   362livefxmarket.com | 520: Web server is returning an unknown error\n  </title>\n  <meta charset="utf-8"/>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>\n  <meta content="noindex, nofollow" name="robots"/>\n  <meta content="width=device-width,initial-scale=1" name="viewport"/>\n  <link href="/cdn-cgi/styles/main.css" id="cf_styles-css" rel="stylesheet"/>\n </head>\n <body>\n  <div id="cf-wrapper">\n   <div class="p-0" id="cf-error-details">\n    <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">\n     <h1 class="inline-block sm:block sm:mb-2 font

In [296]:
## Define function that scrapes through frames for a "lang" attribute.

def find_language(sample_url):
    print(sample_url)
    lang_result="None found."

    ## Using Chrome to access web
    driver = webdriver.Chrome()

    # Open the website
    try:
        driver.get(sample_url)
    except:
        print("Couldn't access site.")
        lang_result="Site inaccessible"
        return lang_result

    # Add delay
    driver.implicitly_wait(30)

    # Check original frame

    found=0
    #driver.switch_to.default_content()
    try:
        lang_tag=driver.find_element("tag name","html")
        lang_result=lang_tag.get_attribute("lang")
        found=1
    except:
        found=0

    # Cycle frames

    frame=0
    while((frame<10) & (found==0)):
        print(frame)
        try:
            driver.switch_to.frame(frame)
            lang_tag=driver.find_element("tag name","html")
            lang_result=lang_tag.get_attribute("lang")
            found=1
        except:
            frame=frame+1

    print(lang_result)
    return lang_result

In [264]:
## Lang attribute scraping: part 1

domain_languages=[]
for x in domains.company_website[1:]:
    domain_languages.append(find_language(x))

http://www.firstamericansecurities.com
Couldn't access site.
http://www.1stnewyorkfinancial.com    
Couldn't access site.
 205fx.com
Couldn't access site.
www.24providenttrade.com
Couldn't access site.
Not found.
Couldn't access site.
www.247cryptotrade.co 
Couldn't access site.
https://www.24brickstonefx.com 
Couldn't access site.
https://24cryptoindex.com/index-2.html

 https://24tradeconnect.com
Couldn't access site.
 http://24tradespace.com
Couldn't access site.
www.360cryptominners.com
Couldn't access site.
 www.360traders.co
Couldn't access site.
https://362livefxmarket.com
en
www.365tradeposition.com
Couldn't access site.
www.426investmentsinc.com
Couldn't access site.
https://9figuretradechannel.com/
Couldn't access site.
www.abacusglobalgroup.com
Couldn't access site.
http://www.accerma.com
Couldn't access site.
http://www.abrokercorp.com/
Couldn't access site.
https://access-financial.com
Couldn't access site.
None found.
Couldn't access site.
 www.accruedequities.trade
Could

UnexpectedAlertPresentException: Alert Text: Wealthy people invest first and spend what’s left and broke people spend first and invest what’s left.
Message: unexpected alert open: {Alert text : Wealthy people invest first and spend what’s left and broke people spend first and invest what’s left.}
  (Session info: chrome=108.0.5359.124)
Stacktrace:
0   chromedriver                        0x0000000105a34f38 chromedriver + 4910904
1   chromedriver                        0x00000001059b4a03 chromedriver + 4385283
2   chromedriver                        0x00000001055f9747 chromedriver + 472903
3   chromedriver                        0x000000010567f0ce chromedriver + 1020110
4   chromedriver                        0x0000000105663ee3 chromedriver + 909027
5   chromedriver                        0x000000010562e30c chromedriver + 688908
6   chromedriver                        0x000000010562f88e chromedriver + 694414
7   chromedriver                        0x0000000105a021de chromedriver + 470268

In [282]:
## Testing output

print(len(domain_languages))
print(domains.iloc[1276])
domain_languages[-1]

1275
company_name                            Traderex Mining
company_website      https://traderexminingexchange.com
sec_category       [ Unregistered Soliciting Entities ]
Name: 1276, dtype: object


'en'

In [291]:
## Adjusting list manually

partial=["zh-cn"]
print(partial)
domain_languages=partial+domain_languages
len(domain_languages)

['zh-cn']


1276

In [299]:
## Adjust list again

domain_languages.append("en-us")
domain_languages[-1]

'en-us'

In [300]:
## Lang attribute scraping: part 2

for x in domains.company_website[1277:]:
    domain_languages.append(find_language(x))


http://www.tradersfinancialinsurance.com
Couldn't access site.
http://www.tradersroom.com/
en
https://traderyt.com
en-US
https://tradesbail247.com
Couldn't access site.
www.tradevilley.org
Couldn't access site.
 www.tradingventurefsi.com
Couldn't access site.
http://www.trans-atlanticalliance.com/
Couldn't access site.
http://www.transfersecurities.com
en
http://treadstonepe.com/
Couldn't access site.
http://trendlinefutures.com/
en
http://www.triasltd.com/
Couldn't access site.
http://tribune-assets.com/
Couldn't access site.
http://www.tridentcapmgt.com/
Couldn't access site.
http://www.trident-ts.com/
en-GB
http://trilliumadvisory.com
en
http://triumphcapitalmgmt.com/
Couldn't access site.
http://trumanst.com/
Couldn't access site.
Not found.
Couldn't access site.
https://ttbdtraders.com

http://www.tudorcapitalgroup.com
en
http://www.tlc-law.com
Couldn't access site.
http://tsc-law.com
en-US
 www.twitxoptions.com
Couldn't access site.
http://www.bma-us.org
Couldn't access site.
www

In [304]:
## Add language column to pandas dataframe

domains["site_language"]=domain_languages
domains.head()

,company_name,company_website,sec_category,site_language
0,12 West Capital Management,http://www.12-wcm.com,[ Impersonators of Genuine Firms ],zh-cn
1,1st American Securities,http://www.firstamericansecurities.com,[ Unregistered Soliciting Entities ],Site inaccessible
2,1st New York Financial Corporation,http://www.1stnewyorkfinancial.com,[ Unregistered Soliciting Entities ],Site inaccessible
3,205fx.com,205fx.com,[ Unregistered Soliciting Entities ],Site inaccessible
4,24 Provident Trade,www.24providenttrade.com,[ Unregistered Soliciting Entities ],Site inaccessible


In [329]:
## Clean dataset

domains["company_name_lower"]=domains.company_name.str.lower()
domains.head()

,company_name,company_website,sec_category,site_language,company_name_lower
0,12 West Capital Management,http://www.12-wcm.com,[ Impersonators of Genuine Firms ],zh-cn,12 west capital management
1,1st American Securities,http://www.firstamericansecurities.com,[ Unregistered Soliciting Entities ],Site inaccessible,1st american securities
2,1st New York Financial Corporation,http://www.1stnewyorkfinancial.com,[ Unregistered Soliciting Entities ],Site inaccessible,1st new york financial corporation
3,205fx.com,205fx.com,[ Unregistered Soliciting Entities ],Site inaccessible,205fx.com
4,24 Provident Trade,www.24providenttrade.com,[ Unregistered Soliciting Entities ],Site inaccessible,24 provident trade


In [318]:
domains[domains.company_website!="Not found."]

,company_name,company_website,sec_category,site_language
0,12 West Capital Management,http://www.12-wcm.com,[ Impersonators of Genuine Firms ],zh-cn
1,1st American Securities,http://www.firstamericansecurities.com,[ Unregistered Soliciting Entities ],Site inaccessible
2,1st New York Financial Corporation,http://www.1stnewyorkfinancial.com,[ Unregistered Soliciting Entities ],Site inaccessible
3,205fx.com,205fx.com,[ Unregistered Soliciting Entities ],Site inaccessible
4,24 Provident Trade,www.24providenttrade.com,[ Unregistered Soliciting Entities ],Site inaccessible
...,...,...,...,...
1443,Zahamax,https://zahamax.com,[ Unregistered Soliciting Entities ],en
1444,Zenith Research Group,http://www.zenithresearchgroup.com/,[ Unregistered Soliciting Entities ],Site inaccessible
1445,Zenithmine,https://zenithmine.net,[ Unregistered Soliciting Entities ],en-US
1446,Zeus Financial Trades,https://zeusfinancialtrades.com,[ Impersonators of Genuine Firms ],Site inaccessible


## Twitter playground

In [29]:
## Using Chrome to access web
driver = webdriver.Chrome()

# Open the website
driver.get(click_links[0])

# Add delay
driver.implicitly_wait(30)

In [ ]:
# Find the username box
name_box = driver.find_element("name","user")

# Send username information
name_box.send_keys(username)

# # Find the format box
# format_box = driver.find_element("name", "format")

# # Send format information
# format_box.send_keys(game_type)

# Find and click search button
search=driver.find_element("tag name","button")
search.click()

In [319]:
## Set authorization keys and constant variables. 

config = dotenv.dotenv_values(".env")
[print(key) for key in config]
MY_HANDLE="@nk_china"

TWTR_API_KEY
TWTR_API_KEY_SECRET
TWTR_BEARER_TOKEN
TEST_ACCESS_TOKEN
TEST_ACCESS_TOKEN_SECRET


Available fields: 

![image1](twitter_fields.jpg)



In [320]:
## Authentication data

consumer_key=config["TWTR_API_KEY"]
consumer_secret=config["TWTR_API_KEY_SECRET"]
access_token=config["TEST_ACCESS_TOKEN"]
access_token_secret=config["TEST_ACCESS_TOKEN_SECRET"]
bearer_token=config["TWTR_BEARER_TOKEN"]


### v1

In [17]:
## Setting up Tweepy. Code template provided by https://towardsdatascience.com/how-to-extract-data-from-the-twitter-api-using-python-b6fbd7129a33

# auth = tweepy.OAuth1UserHandler(
#   consumer_key, 
#   consumer_secret, 
#   access_token, 
#   access_token_secret
# )

# api = tweepy.API(auth)

# tweets = api.search_tweets("covid", lang="zh",tweet_mode="extended", count=5)

### v2

In [321]:
## Setting up Tweepy.

client2 = tweepy.Client(bearer_token)

In [5]:
## Tweepy f

In [322]:
## Find MOFA Twitter handle

mofa_id=client2.get_user(username="LumunChina").data["id"]

In [323]:
## Get MOFA tweets.

client2.get_users_tweets(id=mofa_id, max_results=5, expansions=["geo.place_id"]).data


[<Tweet id=1212970980420804608 text='The time for me political peace is now. China will use its diplomatic channels to bring the parties to the negotiating table'>,
 <Tweet id=1212633552023633921 text='It is time for the security council to Unite for Peace!'>,
 <Tweet id=1212619206044327938 text='The UK has taken unilateral action, threatening the security of Libya. China is suspicious of its intentions but is glad to see the Security Council condemning such action.'>,
 <Tweet id=1212610356130004992 text='China is proud to see peace talks being held for Libya. We have always supported a multilateral discussion and are glad to see the Security Council has realized its importance'>,
 <Tweet id=1212247079092928513 text='China will launch an investigation into the murder of our workers in Mirpur. We will carry out out a joint investigation with Pakistan.'>]

In [ ]:
## Generate the SEC list




In [339]:
## Method for getting recent tweets about COVID.

def get_recent_tweets(user_query, client):
    tweets=client.search_recent_tweets(query=user_query,tweet_fields=["created_at","lang","public_metrics"],max_results=10)
    try:
        for tweet in tweets.data:
            print(tweet["text"], "\n")
            print(tweet["created_at"],"\n")
            print(tweet["lang"],"\n")
            print(tweet["public_metrics"], "\n")
            print("===========")
    except:
        print("No tweets found.")
    return  

In [340]:
get_recent_tweets("as;ld",client2)

RT @louise_marwick: I always think it’s a shame that as LD or MH nurses with a lot of medical skills &amp; knowledge we are unable to help in t… 

2023-01-04 00:01:19+00:00 

en 

{'retweet_count': 6, 'reply_count': 0, 'like_count': 0, 'quote_count': 0} 

RT @louise_marwick: I always think it’s a shame that as LD or MH nurses with a lot of medical skills &amp; knowledge we are unable to help in t… 

2023-01-03 19:46:25+00:00 

en 

{'retweet_count': 6, 'reply_count': 0, 'like_count': 0, 'quote_count': 0} 

RT @louise_marwick: I always think it’s a shame that as LD or MH nurses with a lot of medical skills &amp; knowledge we are unable to help in t… 

2023-01-03 19:40:07+00:00 

en 

{'retweet_count': 6, 'reply_count': 0, 'like_count': 0, 'quote_count': 0} 

RT @louise_marwick: I always think it’s a shame that as LD or MH nurses with a lot of medical skills &amp; knowledge we are unable to help in t… 

2023-01-03 19:39:44+00:00 

en 

{'retweet_count': 6, 'reply_count': 0, 'like_count':

In [364]:
## Do a quick loop for some shiz
partial_domains=domains.iloc[100:120]
# print(partial_domains.iloc[0].company_website)
# get_recent_tweets(partial_domains.iloc[0].company_website,client2)

In [368]:
domains[domains.company_name=="IOU Financial"]

,company_name,company_website,sec_category,site_language,company_name_lower


In [374]:
for x in range(len(partial_domains)):
    print(partial_domains.iloc[x].company_name)
    print("******************")
    get_recent_tweets(partial_domains.iloc[x].company_name_lower,client2)
    get_recent_tweets(partial_domains.iloc[x].company_name,client2)
    try:
        get_recent_tweets(partial_domains.iloc[x].company_website,client2)
    except:
        print("No website matches")
        pass

Atlantic Group
******************


BadRequest: 400 Bad Request
There were errors processing your request: no viable alternative at character '.' (at position 24), no viable alternative at input '[' (at position 21)

## Hoaxy API Sample

## Weibo Playground

In [ ]:
## Try to bring in some stuff from Weibo

